# Shallow detectors

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
drivepath = '/content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/DL_Group_Project'
datapath = '/content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/dl-group-project-code/preprocessed_data'
output_path = f'{drivepath}/experiments/phoneme_detectors_context/'

In [3]:
!pip install tqdm

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
from tqdm import tqdm
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd

In [37]:
CONTEXT = 5
NUM_EPOCHS = 50
BATCH_SIZE = 64
HIDDEN_SIZE = 128
MODEL_VERSION = 1
LEARNING_RATE = 0.01
LOGISTIC_THRESHOLD = 0.5
OTHER_PHONEMES_PERCENT = 0.1

In [38]:
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0
DEVICE = "cuda" if cuda else "cpu"
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

Cuda = True with num_workers = 8


In [59]:
# NOTE: because we have context, we cannot just discard silence frames from the
# other recordings. We will include frames from other recordings, up to the 
# percent specified in the hyperparameter
class PhonemeDataset(Dataset):
    
    def __init__(self, basepath, phoneme_tag, mode, context_size):
      self.context_size = context_size

      # AA.wav
      # AA recording -> FRAMES, LABELS
      # 70% of FRAMES to train; 20% to dev; 10% to test
      # AA_train_features.npy (NUM_FRAMES, 40)
      # AA_dev_features.npy
      # AA_test_features.npy
      # AA_train_labels.npy

      # load specific recording features and labels
      phoneme_features_path = f"{basepath}/{phoneme_tag}_{mode}_features.npy"
      phoneme_labels_path = f"{basepath}/{phoneme_tag}_{mode}_labels.npy"

      phoneme_features = np.load(phoneme_features_path, allow_pickle=True)
      phoneme_labels = np.load(phoneme_labels_path, allow_pickle=True)

      print(f"[{phoneme_tag}] phoneme_features.shape: {phoneme_features.shape}")
      print(f"[{phoneme_tag}] phoneme_labels.shape: {phoneme_labels.shape}")

      phoneme_labels = np.where(phoneme_labels != 0, 1, 0)  # change labels to 0 and 1

      # load % of of other recordings, assign them label = 0
      # go through all other files in basepath
      with os.scandir(basepath) as entries:
        for entry in entries:
          if entry.is_file() and phoneme_tag not in entry.name:
            if "features" in entry.name and mode in entry.name:
              other_phoneme_tag = entry.name.split("_")[0]

              features_filepath = entry.path
              labels_filepath = f"{basepath}/{other_phoneme_tag}_{mode}_labels.npy"

              other_phoneme_features = np.load(features_filepath, allow_pickle=True)
              other_phoneme_labels = np.load(labels_filepath, allow_pickle=True)
              
              end_index = int(OTHER_PHONEMES_PERCENT * other_phoneme_features.shape[0])

              # stack to phoneme features
              phoneme_features = np.concatenate((phoneme_features, other_phoneme_features[:end_index]))
              phoneme_labels = np.concatenate((phoneme_labels, np.zeros(other_phoneme_labels[:end_index].shape)))
              

      print(f"[{phoneme_tag}] complete list phoneme_features.shape: {phoneme_features.shape}")
      print(f"[{phoneme_tag}] complete list phoneme_labels.shape: {phoneme_labels.shape}")

      self.X = torch.Tensor(phoneme_features).float()
      self.y = torch.Tensor(phoneme_labels).float()

    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, index):
        # if dealing with first indexes
        if index - self.context_size <= 0:
          padding_left = self.context_size - index
          padding_left_tensor = torch.zeros(40 * padding_left)
          x_with_target = self.X[0:index + self.context_size + 1].float().reshape(-1)
          x = torch.hstack((padding_left_tensor, x_with_target))
        elif index + self.context_size >= len(self.y):  # if dealing with last indexes
          padding_right = index + self.context_size - len(self.y)
          padding_right_tensor = torch.zeros(40 * padding_right)
          x_with_target = self.X[index - self.context_size - 1:].float().reshape(-1)
          x = torch.hstack((x_with_target, padding_right_tensor))
        else:
          x = self.X[index - self.context_size:index + self.context_size + 1].float().reshape(-1)

        y = self.y[index]
    
        return x,y

In [60]:
class SilenceDataset(Dataset):
    
    def __init__(self, basepath, mode, context_size):
      self.context_size = context_size

      phoneme_features = np.zeros((1, 40))  # eliminate this row
      phoneme_labels = np.zeros((1))  # eliminate this row

      # go through all other files in basepath and load phoneme=1 as label=0 
      # (non-silence) and phoneme=0 as label=1 (silence)
      with os.scandir(basepath) as entries:
        for entry in entries:
          if entry.is_file() and phoneme_tag not in entry.name:
            if "features" in entry.name and mode in entry.name:
              other_phoneme_tag = entry.name.split("_")[0]

              features_filepath = entry.path
              labels_filepath = f"{basepath}/{other_phoneme_tag}_{mode}_labels.npy"

              other_phoneme_features = np.load(features_filepath, allow_pickle=True)
              other_phoneme_labels = np.load(labels_filepath, allow_pickle=True)
              
              # find frames where label != 0
              zero_indexes = np.where(other_phoneme_labels == 0)
              non_zero_indexes = other_phoneme_labels.nonzero()

              other_phoneme_labels[zero_indexes] = 1  # label=1 (silence)
              other_phoneme_labels[non_zero_indexes] = 0  # label=0 (phoneme)
              
              # stack to phoneme features
              phoneme_features = np.concatenate((phoneme_features, other_phoneme_features))
              phoneme_labels = np.concatenate((phoneme_labels, other_phoneme_labels))
              
      self.X = torch.Tensor(phoneme_features[1:]).float()
      self.y = torch.Tensor(phoneme_labels[1:]).float()

    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, index):

        # if dealing with first indexes
        if index - self.context_size <= 0:
          padding_left = self.context_size - index
          padding_left_tensor = torch.zeros(40 * padding_left)
          x_with_target = self.X[0:index + self.context_size + 1].float().reshape(-1)
          x = torch.hstack((padding_left_tensor, x_with_target))
        elif index + self.context_size >= len(self.y):  # if dealing with last indexes
          padding_right = index + self.context_size - len(self.y)
          padding_right_tensor = torch.zeros(40 * padding_right)
          x_with_target = self.X[index - self.context_size - 1:].float().reshape(-1)
          x = torch.hstack((x_with_target, padding_right_tensor))
        else:
          x = self.X[index - self.context_size:index + self.context_size + 1].float().reshape(-1)

        y = self.y[index]
    
        return x,y

In [61]:
def make_dataloader(dataset, train, batch_size):
  if train:
    shuffle = True
    drop_last = True
  else:
    shuffle = False
    drop_last = False
    
  loader = DataLoader(dataset=dataset, batch_size=batch_size,
                      drop_last=drop_last, shuffle=shuffle,
                      pin_memory=True, num_workers=8)
  
  return loader

In [62]:
class PhonemeShallowDetector(nn.Module):
  
  def __init__(self, input_size, hidden_size, activation):
    super(PhonemeShallowDetector, self).__init__()
    
    self.linear_layer = nn.Linear(in_features=input_size, out_features=hidden_size)
    self.bn_layer = nn.BatchNorm1d(num_features=hidden_size)
    self.activation = activation
    self.output_layer = nn.Linear(in_features=hidden_size, out_features=1)
    self.sigmoid = nn.Sigmoid()
    seq_params = [
      self.linear_layer,
      self.bn_layer,
      self.activation,
      self.output_layer,
      self.sigmoid
    ]

    self.network = nn.Sequential(*seq_params)
    
  def forward(self, x):
    return self.network(x)

In [63]:
class ShallowDetector():

  def __init__(self, phoneme_tag, context_size):
    self.phoneme_tag = phoneme_tag

    if phoneme_tag == 'SIL':
      train_data = SilenceDataset(basepath=datapath, mode="train", context_size=context_size)
      self.train_loader = make_dataloader(dataset=train_data, train=True, batch_size=BATCH_SIZE)
      print(f"[{phoneme_tag}] train_data.shape: {train_data.X.shape}")

      dev_data = SilenceDataset(basepath=datapath, mode="dev", context_size=context_size)
      self.dev_loader = make_dataloader(dataset=dev_data, train=False, batch_size=BATCH_SIZE)
      print(f"[{phoneme_tag}] dev_data.shape: {dev_data.X.shape}")
    else:
      train_data = PhonemeDataset(basepath=datapath, phoneme_tag=phoneme_tag, mode="train", context_size=context_size)
      self.train_loader = make_dataloader(dataset=train_data, train=True, batch_size=BATCH_SIZE)
      print(f"[{phoneme_tag}] train_data.shape: {train_data.X.shape}")

      dev_data = PhonemeDataset(basepath=datapath, phoneme_tag=phoneme_tag, mode="dev", context_size=context_size)
      self.dev_loader = make_dataloader(dataset=dev_data, train=False, batch_size=BATCH_SIZE)
      print(f"[{phoneme_tag}] dev_data.shape: {dev_data.X.shape}")

    # when we have context, the input size will be 40*K + 40 + 40*K
    self.model = PhonemeShallowDetector(input_size=(context_size * 2 + 1)*40,
                                        hidden_size=HIDDEN_SIZE, 
                                        activation=nn.LeakyReLU()).to(DEVICE)
    self.criterion = nn.BCELoss()
    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=LEARNING_RATE, momentum=0.9)
    self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min')

    self.train_loss_per_epoch = []
    self.train_acc_per_epoch = []
    self.dev_loss_per_epoch = []
    self.dev_acc_per_epoch = []
  
  def save_model(self, epoch):
    model_epoch_path = "{}/models/model_{}_{}_{}".format(output_path, 
                                                         self.phoneme_tag, 
                                                         MODEL_VERSION, 
                                                         epoch)
    torch.save({
        'model_state_dict': self.model.state_dict(),
        'optimizer_state_dict': self.optimizer.state_dict(),
        'scheduler_state_dict': self.scheduler.state_dict(),
    }, model_epoch_path)
    # print('saved model: {}'.format(model_epoch_path))

  def train(self, epochs):
    # Run training and track with wandb
    total_batches = len(self.train_loader) * epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0

    for epoch in tqdm(range(epochs)):
        train_loss = 0.0
        start_time = time.time()
        total_predictions = 0
        correct_predictions = 0

        true_labels = []
        predictions = []
        for _, (features, targets) in enumerate(self.train_loader):
            batch_loss, outputs = self.train_batch(features, targets)
            train_loss += batch_loss

            example_ct += len(features)
            batch_ct += 1

            targets = targets.reshape(-1, 1)

            # check number of correct predictions
            output_classes = torch.where(outputs > LOGISTIC_THRESHOLD, 1, 0).detach().cpu() # convert to class labels
            total_predictions += len(output_classes)
            correct_predictions += torch.sum(targets == output_classes)

            true_labels += list(targets)
            predictions += list(output_classes)

        end_time = time.time()

        train_loss /= example_ct
        print(f"training loss: {train_loss}; time: {end_time - start_time}s")
        
        if (epoch + 1) % 10 == 0 or epoch == (epochs - 1):
          report = classification_report(true_labels, predictions, output_dict=True)
          df = pd.DataFrame(report).transpose()
          df.to_csv(f"{output_path}/reports/reports_train_{self.phoneme_tag}_{MODEL_VERSION}_{epoch + 1}.csv", index=False)

        print(classification_report(true_labels, predictions))
        print(confusion_matrix(true_labels, predictions))

        self.train_loss_per_epoch.append(train_loss)
        #self.train_acc_per_epoch.append(train_acc)

        # evaluate model with validation data
        dev_loss = self.evaluate_model(epoch)
        
        self.dev_loss_per_epoch.append(dev_loss)
        #self.dev_acc_per_epoch.append(dev_acc)

        # Step with the scheduler
        self.scheduler.step(dev_loss)
      
    # epoch completed, save model
    self.save_model(epoch)

  def train_batch(self, features, targets):
    features, targets = features.to(DEVICE), targets.to(DEVICE)
    targets = targets.reshape(-1, 1)

    self.optimizer.zero_grad()

    # Forward pass ➡
    outputs = self.model(features)
    loss = self.criterion(outputs, targets)  # compare with target outputs
    # Backward pass ⬅
    loss.backward()
    # Step with optimizer
    self.optimizer.step()

    return loss.item(), outputs

  def evaluate_model(self, epoch):

    with torch.no_grad():
      self.model.eval()

      running_loss = 0.0
      total_predictions = 0.0
      correct_predictions = 0.0

      true_labels = []
      predictions = []

      example_ct = 0
      start_time = time.time()
      for batch_idx, (features, targets) in enumerate(self.dev_loader):
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
        targets = targets.reshape(-1, 1)

        example_ct += len(features)

        outputs = self.model(features)
        outputs = outputs.to(DEVICE)

        # check number of correct predictions
        output_classes = torch.where(outputs > LOGISTIC_THRESHOLD, 1, 0)  # convert to class labels
        total_predictions += len(output_classes)
        correct_predictions += torch.sum(targets == output_classes)

        loss = self.criterion(outputs, targets).detach()
        running_loss += loss.item()

        true_labels += list(targets.detach().cpu())
        predictions += list(output_classes.detach().cpu())
      
      end_time = time.time()

      running_loss /= example_ct
      print(f"testing loss: {running_loss}; time: {end_time - start_time}s")

      if (epoch + 1) % 10 == 0:
        report = classification_report(true_labels, predictions, output_dict=True)
        df = pd.DataFrame(report).transpose()
        df.to_csv(f"{output_path}/reports/reports_dev_{self.phoneme_tag}_{MODEL_VERSION}_{epoch + 1}.csv", index=False)
  
      print(classification_report(true_labels, predictions))
      print(confusion_matrix(true_labels, predictions))
      return running_loss

## 2. Training shallow detectors (per phoneme + one for silence)

In [64]:
%cd /content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/dl-group-project-code/utilities

/content/gdrive/MyDrive/Spring_2021/11785_Intro_to_Deep_Learning/dl-group-project-code/utilities


In [65]:
from utilities import PHONEME_MAPPER

In [66]:
%cd /

/


In [67]:
print(PHONEME_MAPPER)

{0: 'SIL', 1: 'AE', 2: 'AH', 3: 'AW', 4: 'AY', 5: 'B', 6: 'EH', 7: 'D', 8: 'DH', 9: 'EE', 10: 'FF', 11: 'G', 12: 'HH', 13: 'IH', 14: 'II', 15: 'J', 16: 'K', 17: 'LL', 18: 'MM', 19: 'NN', 20: 'OH', 21: 'OO', 22: 'OW', 23: 'OY', 24: 'P', 25: 'RR', 26: 'SH', 27: 'SS', 28: 'T', 29: 'TH', 30: 'UE', 31: 'UH', 32: 'VV', 33: 'WW', 34: 'YY', 35: 'ZZ', 36: 'CH', 37: 'ER', 38: 'NG'}


In [68]:
for phoneme_index, phoneme_tag in PHONEME_MAPPER.items():
  detector = ShallowDetector(phoneme_tag, CONTEXT)
  detector.train(epochs=NUM_EPOCHS)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/50 [00:00<?, ?it/s]

[SIL] train_data.shape: torch.Size([54579, 40])
[SIL] dev_data.shape: torch.Size([11712, 40])
training loss: 0.00287145158100929; time: 2.7909927368164062s
              precision    recall  f1-score   support

         0.0       0.96      0.87      0.91     21519
         1.0       0.92      0.97      0.95     33009

    accuracy                           0.93     54528
   macro avg       0.94      0.92      0.93     54528
weighted avg       0.94      0.93      0.93     54528

[[18774  2745]
 [  832 32177]]
testing loss: 0.004118349096518967; time: 0.6313862800598145s
              precision    recall  f1-score   support

         0.0       0.90      0.87      0.89      5314
         1.0       0.89      0.92      0.91      6398

    accuracy                           0.90     11712
   macro avg       0.90      0.90      0.90     11712
weighted avg       0.90      0.90      0.90     11712



  2%|▏         | 1/50 [00:05<04:26,  5.45s/it]

[[4619  695]
 [ 504 5894]]
training loss: 0.001867470935326203; time: 2.7082090377807617s
              precision    recall  f1-score   support

         0.0       0.95      0.84      0.89     21517
         1.0       0.90      0.97      0.94     33011

    accuracy                           0.92     54528
   macro avg       0.93      0.91      0.91     54528
weighted avg       0.92      0.92      0.92     54528

[[18055  3462]
 [  924 32087]]
testing loss: 0.004515726486019933; time: 0.6228275299072266s
              precision    recall  f1-score   support

         0.0       0.95      0.79      0.87      5314
         1.0       0.85      0.97      0.90      6398

    accuracy                           0.89     11712
   macro avg       0.90      0.88      0.88     11712
weighted avg       0.90      0.89      0.89     11712



  4%|▍         | 2/50 [00:10<04:20,  5.43s/it]

...

testing loss: 0.005750258798659956; time: 1.0326969623565674s
              precision    recall  f1-score   support

         0.0       0.99      0.89      0.94      1217
         1.0       0.46      0.94      0.62       124

    accuracy                           0.89      1341
   macro avg       0.73      0.91      0.78      1341
weighted avg       0.94      0.89      0.91      1341

[[1083  134]
 [   8  116]]
training loss: 3.7723541942869254e-06; time: 1.2454516887664795s
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5842
         1.0       0.99      0.96      0.98       430

    accuracy                           1.00      6272
   macro avg       0.99      0.98      0.99      6272
weighted avg       1.00      1.00      1.00      6272

[[5837    5]
 [  16  414]]


 98%|█████████▊| 49/50 [02:04<00:02,  2.54s/it]

testing loss: 0.005735563938090985; time: 1.0612759590148926s
              precision    recall  f1-score   support

         0.0       0.99      0.89      0.94      1217
         1.0       0.46      0.94      0.62       124

    accuracy                           0.89      1341
   macro avg       0.73      0.91      0.78      1341
weighted avg       0.94      0.89      0.91      1341

[[1083  134]
 [   8  116]]
training loss: 3.704929496627895e-06; time: 1.2510714530944824s
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5842
         1.0       0.99      0.97      0.98       430

    accuracy                           1.00      6272
   macro avg       0.99      0.98      0.99      6272
weighted avg       1.00      1.00      1.00      6272

[[5837    5]
 [  14  416]]


100%|██████████| 50/50 [02:06<00:00,  2.54s/it]

testing loss: 0.005759074397259764; time: 1.0405495166778564s
              precision    recall  f1-score   support

         0.0       0.99      0.89      0.94      1217
         1.0       0.46      0.94      0.62       124

    accuracy                           0.89      1341
   macro avg       0.73      0.91      0.78      1341
weighted avg       0.94      0.89      0.91      1341

[[1083  134]
 [   8  116]]
